# Deep Learning
## Optimization

Author: Bingchen Wang

Last Updated: 13 Nov, 2022


<nav>
    <b>Deep learning navigation:</b> <a href="./Deep Learning Basics.ipynb">Basics</a>
</nav>

---
<nav>
    <a href="../Machine%20Learning.ipynb">Machine Learning</a> |
    <a href="../Supervised Learning/Supervised%20Learning.ipynb">Supervised Learning</a>
</nav>

---

## Contents
Algorithms:
- [Batch Gradient Descent](#BGD)
- [Stochastic Gradient Descent](#SGD)
- [Mini-batch Gradient Descent](#MBGD)
- [Gradient Descent with Momentum](#GDwM)
- [RMSprop](#RMSp)
- [Adam](#ADAM)

Technique:
- [Learning Rate Decay](#LRD)
- [Batch Normalization](#BN)
    - [PyTorch Implementation](./PyTorch/Batch%20Norm.ipynb)

## Algorithms
Consider the linear model/hypothesis
$$
h_{\theta}(x^{(i)}) = \theta^{\mathsf{T}} x^{(i)}
$$
and the squared loss function
$$
L(\theta) = \frac{1}{2} {\left(y^{(i)} - h_{\theta}(x^{(i)})\right)}^2
$$

<a name = "BGD"></a>
### Batch Gradient Descent
Use the entire training set of $m$ examples in every iteration of the training process.
<blockquote>
    Start with some $\theta \in \mathbb{R}^p$ <br>
    Repeat until convergence:
    <blockquote>
        Compute $h_\theta(x^{(i)})$ for <b>all</b> examples $i = 1, \dots, m$. <br>
        Calculate the cost $$J(\theta) = \frac{1}{2m} \sum_{i=1}^m {\left(y^{(i)} - h_{\theta}(x^{(i)})\right)}^2.$$ <br>
        Update the parameters $$\theta_j := \theta_j - \alpha \frac{\partial J}{\partial \theta_j}, \;\;\; \text{for } j = 1, \dots, P$$
    </blockquote>
</blockquote>    

<a name ="SGD"></a>
### Stochastic Gradient Descent
Use a single example in every iteration of the training process.

<blockquote>
    Start with some $\theta \in \mathbb{R}^p$ <br>
    Repeat until an approximate minimum is obtained or a certain threshold is met (e.g., max numbers of epochs):
    <blockquote>
        Randomly shuffle examples in the training set. <br>
        For $i = 1, \dots, m$ do:
        <blockquote>
            Compute $h_\theta(x^{(i)})$. <br>
            Calculate the cost $$J(\theta) = \frac{1}{2} {\left(y^{(i)} - h_{\theta}(x^{(i)})\right)}^2.$$ <br>
            Update the parameters $$\theta_j := \theta_j - \alpha \frac{\partial J}{\partial \theta_j}, \;\;\; \text{for } j = 1, \dots, P$$
        <blockquote>
    </blockquote>
</blockquote>    

<a name = "MBGD"></a>
### Mini-batch Gradient Descent
Use a subset (a mini-batch) of the training set in every iteration of the training process. Denote the size of a mini-batch as $m_t$ and the total number of mini-batches as $T$, such that:
$$
m = m_t T
$$
<div class = "alert alert-block alert-info"> Typical choices for $m_t$: 64, 128, 256, 512, 1024. (Make sure that the mini-batch fits in the CPU/GPU memory.)
</div>
Further, denote the data in the mini-batch $t = 1, \dots, T$ as $\{X^{\{t\}}, y^{\{t\}}\}$.

<blockquote>
    Start with some $\theta \in \mathbb{R}^p$ <br>
    Repeat until an approximate minimum is obtained or a certain threshold is met (e.g., max numbers of epochs):
    <blockquote>
        Randomly shuffle examples in the training set. Split the training set into $T$ mini-batches.<br>
        For $t = 1, \dots, T$ do:
        <blockquote>
            Compute $h_\theta(x^{\{t\}(i)})$ for all examples in the mini-batch $t$. <br>
            Calculate the cost $$J(\theta) = \frac{1}{2m_t} \sum_{i = 1}^{m_t}{\left(y^{\{t\}(i)} - h_{\theta}(x^{\{t\}(i)})\right)}^2.$$ <br>
            Update the parameters $$\theta_j := \theta_j - \alpha \frac{\partial J}{\partial \theta_j}, \;\;\; \text{for } j = 1, \dots, P$$
        </blockquote>
    </blockquote>
</blockquote>
<div class = "alert alert-block alert-success"><b>Advantages of mini-batch gradient descent</b>:
<ul>
    <li> Make use of vectorization to speed up the training.
    <li> Make progress without processing the entire training set.
</ul>
</div>

#### Performance Comparisions: Batch, Stochastic & Mini-batch Gradient Descents
<div style = "text-align: center;">
    <img src="./images/gradient descent first 100 epochs.png" style="width:60%;" >
    <img src="./images/gradient descent last 10 epochs.png" style="width:60%;" >
</div>

<a name = "GDwM"></a>
### Gradient Descent with Momentum

#### Expoentially Weighted Moving Averages
Denote the original time series examples as $\{x_t\}_{t=1}^T$.

The **exponentially weighted moving averages** are computed using the recursive formula:
$$
\begin{align}
v_0 = & 0 \\
v_t = & \beta v_{t-1} + (1-\beta) x_t, \text{ for } t=1, \dots, T
\end{align}
$$
where $\beta$ is a hyper-parameter that governs the smoothness of the averages. Roughly speaking, $v_t$ can be interpreted as approximately averaging over $\frac{1}{1-\beta}$ days (related to the fact that $(1-\frac{1}{n})^n \approx \frac{1}{e}$, which is considered small enough in contribution to the average).
<div class = "alert alert-block alert-info"> <b>Advantage over simple moving averages/windows:</b> Saves on memory (just need to keep one running variable and keep overwriting it), and computation.<br>
<b>Disadvantage vis-à-vis simple moving averages/windows:</b> not the most accurate way to compute an average.
</div>

A problem with expoentially weighted moving averages is that it requires a **burn-in period** to be able to give sensible estimates of averages, which is conspicuous from the follow equations:
$$
\begin{align}
v_1 = & (1-\beta) x_1 \\
v_2 = & (1-\beta)\beta x_1 + (1-\beta) x_2 \\
v_3 = & (1-\beta)\beta^2 x_1 + (1-\beta)\beta x_2 + (1-\beta) x_3 \\
\cdots &
\end{align}
$$
The closer $\beta$ is to 1, the longer the burn-in period needs to be. To solve this problem and ensure better estimates near the beginning of the time series, bias correction can be used to improve the algorithm:
$$
\tilde v_t = \frac{v_t}{1-\beta^t}, \text{ for } t=1, \dots, T
$$
<div class = "alert alert-block alert-danger"> <b>Note:</b> To apply bias correction, compute the un-corrected series first and then multiply the values of the original uncorrected series by the bias correction terms respectively.
</div>
<div class = "alert alert-block alert-success"> This makes use of the following result in algebra:
    $$
    s_t = \sum_{s = 1}^t (1-\beta) \beta^{s-1} = (1-\beta) \frac{1 - \beta^t}{1-\beta} = 1 - \beta^t
    $$
so
    $$
    \tilde s_t = \frac{s_t}{1-\beta^t} = 1
    $$
</div>

<div style = "text-align: center;">
    <img src="./images/exponentially weighted moving averages.png" style="width:80%;" > <br>
    Applying exponentially weighted moving averages to the Oxford daily temperature data. 
</div>

#### Gradient Descent with Momentum
<blockquote>
    Initiate $v_{d\theta_j} = 0, \text{ for } j = 1, \dots, P$. <br>
    On iteration $t$ (e.g., with mini-batch gradient descent):
    <blockquote>
        Compute $d \theta_j := \frac{\partial J}{\partial \theta_j}, \text{ for } j = 1, \dots, P$ on the current mini-batch. <br>
        Update the momentum terms:
        $$
        v_{d\theta_j} := \beta v_{d\theta_j} + (1-\beta) d \theta_j, \text{ for } j = 1, \dots, P
        $$
        Update the parameter:
        $$
        \theta_j := \theta_j - \alpha v_{d\theta_j}, \text{ for } j = 1, \dots, P
        $$
    </blockquote>
</blockquote>

**Hyperparameters:** $\alpha, \beta$ ($= 0.9$ usually works well; i.e., averaging over the last 10 gradients).
<div class = "alert alert-block alert-info"> <b>Note:</b> Sometimes, the following equivalent updating rule is used:
    $$
    v_{d\theta_j} := \beta v_{d\theta_j} + d \theta_j, \text{ for } j = 1, \dots, P
    $$
where $\alpha$ needs to be re-tuned with a factor of $(1-\beta)$.
</div>

### RMSprop
<blockquote>
    Initiate $s_{d\theta_j} = 0, \text{ for } j = 1, \dots, P$ and $\epsilon = 10^{-8}$. <br>
    On iteration $t$ (e.g., with mini-batch gradient descent):
    <blockquote>
        Compute $d \theta_j := \frac{\partial J}{\partial \theta_j}, \text{ for } j = 1, \dots, P$ on the current mini-batch. <br>
        Update the mean squared term:
        $$
        s_{d\theta_j} := \beta_2 s_{d\theta_j} + (1-\beta_2) d \theta_j^2, \text{ for } j = 1, \dots, P
        $$
        Update the parameter:
        $$
        \theta_j := \theta_j - \alpha \frac{d\theta_j}{\sqrt{s_{d\theta_j} + \epsilon}}, \text{ for } j = 1, \dots, P
        $$
    </blockquote>
</blockquote>

<div class = "alert alert-block alert-success"> <b>Note:</b> We add $\epsilon$ to the denominator to avoid the divide-by-zero problem.
</div>

<a name = "ADAM"></a>
### Adaptive Moment Estimation (Adam)
<blockquote>
    Initiate $v_{d\theta_j} = 0, s_{d\theta_j} = 0, \text{ for } j = 1, \dots, P$ and $\epsilon = 10^{-8}$. <br>
    On iteration $t$ (e.g., with mini-batch gradient descent):
    <blockquote>
        Compute $d \theta_j := \frac{\partial J}{\partial \theta_j}, \text{ for } j = 1, \dots, P$ on the current mini-batch. <br>
        Update the momentum and the mean squared term:
        $$
        \begin{align}
        v_{d\theta_j} := & \beta_1 v_{d\theta_j} + (1-\beta_1) d \theta_j \\
        s_{d\theta_j} := & \beta_2 s_{d\theta_j} + (1-\beta_2) d \theta_j^2
        \end{align}
        $$
        for $j = 1, \dots, P$. <br><br>
        Apply bias corrections:
        $$
        \begin{align}
        v_{d\theta_j}^{\text{corrected}} = & \frac{v_{d\theta_j}}{1-\beta_1^t} \\
        s_{d\theta_j}^{\text{corrected}} = & \frac{s_{d\theta_j}}{1-\beta_2^t}
        \end{align}
        $$
        for $j = 1, \dots, P$. <br><br>
        Update the parameter:
        $$
        \theta_j := \theta_j - \alpha \frac{v_{d\theta_j}^{\text{corrected}}}{\sqrt{s_{d\theta_j}^{\text{corrected}} + \epsilon}}, \text{ for } j = 1, \dots, P
        $$
    </blockquote>
</blockquote>

**Typical values for the hyperparameters:**
- $\alpha$: needs to be tuned
- $\beta_1$ (momentum update): 0.9
- $\beta_2$ (mean squared update): 0.999
- $\epsilon$ : $10^{-8}$

## Technique
<a name = "LRD"></a>
### Learning Rate Decay
Denote the current epoch as $i$, the decay rate as $k$ and the current mini-batch as $t$.

#### Time-based decay
$$
\alpha = \frac{1}{1 + k \times (i-1)} \alpha_0
$$
#### Exponential decay
$$
\alpha = k^{i-1}\alpha_0
$$
#### Inverse square root decay
**Version 1**: Based on the epoch number,
$$
\alpha = \frac{k}{\sqrt{i}} \alpha_0
$$
**Version 2**: Based on the mini-batch number,
$$
\alpha = \frac{k}{\sqrt{t}} \alpha_0
$$
#### Discrete staircase decay
Denote the length of a step as $L_\text{step}$.
$$
\alpha = k^{\left\lfloor \frac{i-1}{L_\text{step}} \right\rfloor}\alpha_0
$$

#### Comparisons of different decay methods
<div style = "text-align: center;">
    <img src="./images/Learning rate decay.png" style="width:60%;" > <br>
</div>

<a name = "BN"></a>
### Batch Normalization
**Recall:** Normalization input features can speed up learning. <br>
**Motivation:** Can we normalize input to each layer so as to make the training of the parameters in each layer faster? <br>
**Effects of batch normalization:**
- Speed up learning
- Make the hyperparameter search problem much easier, with a much bigger range of hyperparameters that work well.
- Have slight regularization effects

#### Batch Normalization
<blockquote>
Given some intermediate values in a neural network $z^{[l](1)}, \dots, z^{[l](m)}$, compute
$$
\begin{align}
\mu = & \frac{1}{m} \sum_i z^{[l](i)} \\
\sigma^2 = & \frac{1}{m} \sum_i {(z^{[l](i)} - \mu)}^2 \\
z^{[l](i)}_{\text{norm}} = & \frac{z^{[l](i)} - \mu}{\sqrt{\sigma^2 +\epsilon}} \\
\tilde z^{[l](i)} = & \gamma z^{[l](i)}_{\text{norm}} + \beta
\end{align}
$$
Then compute $a^{[l]}$ using $\tilde z^{[l]}$ instead of $z^{[l]}$.
</blockquote>

Batch Normalization introduces **two new learnable parameters** to each layer of the model: $\gamma$ and $\beta$, which govern the **variance** and the **mean** of the hidden linear units of the layer.

<div class = "alert alert-block alert-info"> <b>Note:</b> if $\gamma = \sqrt{\sigma^2 +\epsilon} $ and $\beta = \mu$, then
    $$
    \tilde z^{[l](i)} = z^{[l](i)}.
    $$
So by choosing appropriate values of $\gamma$ and $\sigma^2$, this process is essentially computing an identity function. But setting $\gamma$ and $\sigma^2$ to be trainable allows for other means and variances.
</div>

<div class = "alert alert-block alert-info"> <b>Why do we need an extra step, i.e., $\tilde z^{[l](i)} = \gamma z^{[l](i)}_{\text{norm}} + \beta$?</b> <br>
    This allows us to better take advantage of the non-linearity of some activation functions, e.g. sigmoid, instead of having most of the values concentrated on a small region of the (approximately) linear regime.
</div>

What batch norm does essentially is to **standardize** the hidden linear units of a layer to have a **fixed mean** and a fixed **variance**, governed by the parameters $\beta$ and $\gamma$.

<div style = "text-align: center;">
    <img src="./images/Batch normalization in a NN.jpg" style="width:100%;" > <br>
</div>

#### Backprop for Batch Norm
**How does batch norm affect the flow of backpropagation?** 

As is evident from the previous section, batch norm introduces an additional step in the forward propagation from $z^{[l]}$ to $\tilde z^{[l]}$. Thus, during the backprop stage, the key is to figure out **the flow from $d \tilde z^{[l]}$ to $d z^{[l]}$**. A computation graph can be used to solve the puzzle. Yet, here we adopt a *calculus approach*.

To simply notation, we drop the layer labels, and focus on the following process:

$$
\begin{equation}
Z \rightarrow Z_{\text{norm}} \rightarrow \tilde Z \\
\left[\begin{array}{cccc}
z^{(1)}_1 & z^{(2)}_1 & \cdots & z^{(m)}_1 \\
z^{(1)}_2 & z^{(2)}_2 & \cdots & z^{(m)}_2 \\
\vdots & \vdots & \ddots & \vdots \\
z^{(1)}_n & z^{(2)}_n & \cdots & z^{(m)}_n
\end{array}\right] \rightarrow
\left[\begin{array}{cccc}
z^{(1)}_{1,\text{norm}} & z^{(2)}_{1,\text{norm}} & \cdots & z^{(m)}_{1,\text{norm}} \\
z^{(1)}_{2,\text{norm}} & z^{(2)}_{2,\text{norm}} & \cdots & z^{(m)}_{2,\text{norm}} \\
\vdots & \vdots & \ddots & \vdots \\
z^{(1)}_{n,\text{norm}} & z^{(2)}_{n,\text{norm}} & \cdots & z^{(m)}_{n,\text{norm}}
\end{array}\right] \rightarrow
\left[\begin{array}{cccc}
\tilde z^{(1)}_1 & \tilde z^{(2)}_1 & \cdots & \tilde z^{(m)}_1 \\
\tilde z^{(1)}_2 & \tilde z^{(2)}_2 & \cdots & \tilde z^{(m)}_2 \\
\vdots & \vdots & \ddots & \vdots \\
\tilde z^{(1)}_n & \tilde z^{(2)}_n & \cdots & \tilde z^{(m)}_n
\end{array}\right] 
\end{equation}
$$

where each matrix has dimension $(n, m)$–$n$ is the number of features and $m$ the number of examples. Our strategy is to focus on **element-wise gradients**, i.e., $\frac{\partial J}{\partial z^{(j)}_{k}}$, and use the **Chain Rule**.

Recall that
$$
\begin{equation}
z^{(i)}_{k, \text{norm}} = \frac{z^{(i)}_k - \mu_k}{\sqrt{\sigma^2_k + \epsilon}}, \;\;\; \tilde z^{(i)}_k = \gamma_k z^{(i)}_{k, \text{norm}} + \beta_k,\;\;\;
\left\{\begin{array}{l} \mu_k = \frac{1}{m} \sum_{i=1}^m z^{(i)}_k \\ \sigma^2_k = \frac{1}{m} \sum_{i=1}^m {(z^{(i)}_k - \mu_k)}^2 \end{array}\right.
\end{equation}
$$
so **feature $k$ does not make use of any information from feature $p \neq k$**.

Now,
$$
\frac{\partial J}{\partial z^{(j)}_k} = \sum_{i=1}^m \underbrace{\frac{\partial J}{\partial \tilde z^{(i)}_k}}_{\text{given}} \underbrace{\frac{\partial \tilde z^{(i)}_k}{\partial z^{(i)}_{k, \text{norm}}}}_{\gamma_k} \underbrace{\frac{\partial z^{(i)}_{k, \text{norm}}}{\partial z^{(j)}_k}}_{\text{focus here}}
$$

$$
\begin{align}
\frac{\partial z^{(i)}_{k, \text{norm}}}{\partial z^{(j)}_k} = & \frac{1}{\sigma^2_k + \epsilon} 
\left\{
\left(\mathbb{1}_{\{i=j\}} - \frac{1}{m}\right)
\sqrt{\sigma^2_k + \epsilon} + 
(z^{(i)}_k -\mu_k) \frac{1}{2\sqrt{\sigma^2_k + \epsilon}} \frac{1}{m} \sum_{s=1}^m 
\left[2(z^{(s)}_k -\mu_k)\left(\mathbb{1}_{\{s=j\}} - \frac{1}{m}\right)\right]
\right\} \\
 = & \frac{1}{\sqrt{\sigma^2_k + \epsilon}}\left(\mathbb{1}_{\{i=j\}} - \frac{1}{m}\right) + \frac{1}{{(\sigma^2_k + \epsilon)}^{\frac{3}{2}}}\frac{1}{m}(z^{(i)}_k -\mu_k)(z^{(j)}_k -\mu_k)
\end{align}
$$

Therefore,
$$
\begin{align}
\frac{\partial J}{\partial z^{(j)}_k} = & \sum_{i=1}^m \frac{\partial J}{\partial \tilde z^{(i)}_k} \gamma_k \left\{ \frac{1}{\sqrt{\sigma^2_k + \epsilon}}\left(\mathbb{1}_{\{i=j\}} - \frac{1}{m}\right) + \frac{1}{{(\sigma^2_k + \epsilon)}^{\frac{3}{2}}}\frac{1}{m}(z^{(i)}_k -\mu_k)(z^{(j)}_k -\mu_k)\right\} \\
= & \underbrace{\frac{\gamma_k}{\sqrt{\sigma^2_k + \epsilon}} \frac{\partial J}{\partial \tilde z^{(j)}_k}}_{\text{vanila gradient treating $\mu_k$ and $\sigma^2_k$ as given}} + \underbrace{\frac{\gamma_k}{m\sqrt{\sigma^2_k + \epsilon}}  \sum_{i=1}^m \frac{\partial J}{\partial \tilde z^{(i)}_k} \left\{ \frac{(z^{(i)}_k -\mu_k)(z^{(j)}_k -\mu_k)}{\sigma^2_k + \epsilon} -1\right\}}_{\text{additional feedback through $\mu_k$ and $\sigma^2_k$}}
\end{align}
$$

#### Implementing the Backprop for Batch Norm (vectorized version)
Re-write the above equation as:
$$
\begin{align}
\frac{\partial J}{\partial z^{(j)}_k} = & \frac{\gamma_k}{\sqrt{\sigma^2_k + \epsilon}} \frac{\partial J}{\partial \tilde z^{(j)}_k} + \frac{\gamma_k(z^{(j)}_k -\mu_k)}{m(\sigma^2_k + \epsilon)}  \sum_{i=1}^m \frac{\partial J}{\partial \tilde z^{(i)}_k} \frac{(z^{(i)}_k -\mu_k)}{\sqrt{\sigma^2_k + \epsilon}}  - \frac{\gamma_k}{m\sqrt{\sigma^2_k + \epsilon}}  \sum_{i=1}^m \frac{\partial J}{\partial \tilde z^{(i)}_k} \\
\left[\frac{\partial J}{\partial z}\right]_{jk} = & \frac{\gamma_k}{\sqrt{\sigma^2_k + \epsilon}} \left[\frac{\partial J}{\partial \tilde z}\right]_{jk} + \frac{\gamma_k}{\sqrt{\sigma^2_k + \epsilon}} \left[Z_{\text{norm}}\right]_{jk} \cdot \frac{1}{m} \mathsf{np.sum} \left(\left[\frac{\partial J}{\partial \tilde z}\right]_{\cdot k} * \left[Z_{\text{norm}}\right]_{\cdot k} \right) - \frac{\gamma_k}{\sqrt{\sigma^2_k + \epsilon}} \cdot \frac{1}{m} \mathsf{np.sum} \left(\left[\frac{\partial J}{\partial \tilde z}\right]_{\cdot k} \right)
\end{align}
$$

Therefore,
$$
dZ = \frac{\gamma}{\sqrt{\sigma^2 + \epsilon}} d\tilde Z + \frac{\gamma}{\sqrt{\sigma^2 + \epsilon}} Z_{\text{norm}} \cdot \frac{1}{m} \mathsf{np.sum}\left(d\tilde Z * Z_{\text{norm}}, \mathsf{axis = 1} \right) - \frac{\gamma}{\sqrt{\sigma^2 + \epsilon}}  \cdot \frac{1}{m} \mathsf{np.sum}\left(d\tilde Z, \mathsf{axis = 1} \right)
$$

#### Why does Batch Norm work?
1. Similar to normalizing the input features (which makes the cost space less skewed and speeds up gradient descent), Batch Norm normalises values in the hidden units (which are inputs to the next layer).
2. Bath Norm makes the weights of later layers more robust to changes to weights in earlier layers.

#### Regularization effect of Batch Norm
Batch normalization has a slight regularization effect. This is because:
1. During the training time, we **compute the means and variances for each layer using only a single mini-batch at a time**, which  introduces noise to the statistics.
2. Recall that the regularization effect of dropout comes from the **introduction of** multiplicative **noise** (in the form of $\times \; 0$ or $\times \; 1$). The regularization effect of Batch Norm comes from the introduction of both additive noise (from the mean) and multiplicative noise (from the variance).
3. The smaller the variation of means and variances (across the mini-batches), the smaller the noise, and thus the smaller the regularization effect. Therefore, **a larger mini-batch size, say 512 instead of 256, has a smaller regularization effect**.

#### Batch Norm at test time
Need measures of $\mu$ and $\sigma^2$ for each layer to be used at training time. This is usually done by keeping **running averages** (expoentially weighted running averages) of $\mu$ and $\sigma^2$ at the training time. Note that the default approach is to estimate these averages but *not* use them in the training time. If one uses the running averages during the training time, the additional (backprop) feedback through $\mu$ and $\sigma^2$ would be smaller and thus the regularization effect. 